In [ ]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import os
import json
from pathlib import Path

# Configure plotting
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Load Everything and Plot in One Go to Avoid Scope Issues
convergence_traces = []
records = []

dataset_path = Path("../retrieval_dataset_verified.yaml")
if not dataset_path.exists():
    dataset_path = Path("retrieval_dataset_verified.yaml")

if dataset_path.exists():
    print(f"Loading metadata from: {dataset_path}")
    with open(dataset_path, 'r') as f:
        data = yaml.safe_load(f)
    
    cases = data.get('cases', [])
    for case in cases:
        if 'convergence_trace' in case.get('metadata', {}):
            convergence_traces.append({
                'case_id': case['id'],
                'trace': case['metadata']['convergence_trace']
            })
            
        candidates = case.get('candidates', [])
        for ctx in candidates:
            meta = ctx.get('metadata', { })
            records.append({
                'case_id': case['id'],
                'fqn': ctx['fqn'],
                'source_type': ctx['type'],
                'delta_p': meta.get('delta_p', 0.0),
                'n_in': meta.get('n_in', 0),
            })
    stats_df = pd.DataFrame(records)
    
    # Plot 1: Impact Score
    if not stats_df.empty:
        plt.figure(figsize=(12, 6))
        sns.kdeplot(data=stats_df, x='delta_p', hue='source_type', fill=True, common_norm=False)
        plt.title('Impact Score Density by Candidate Source')
        plt.xlabel('Delta P (Impact Score)')
        plt.axvline(0, color='red', linestyle='--')
        plt.show()
        
    # Plot 2: Convergence
    if convergence_traces:
        plt.figure(figsize=(12, 6))
        for trace in convergence_traces:
            y = trace['trace']
            x = range(1, len(y) + 1)
            plt.plot(x, y, alpha=0.5, label=trace['case_id'][:20])

        plt.title('Convergence of Impact Uncertainty over Trials')
        plt.xlabel('Trial Number')
        plt.ylabel('Max Standard Error (Uncertainty)')
        plt.axhline(0.1, color='red', linestyle='--', label='Convergence Threshold (0.1)')
        plt.tight_layout()
        plt.show()
else:
    print("Dataset file not found.")

In [ ]:
# Load Latest Run Logs (YAML Stream)
log_files = glob.glob("../logs/*.yaml")
if not log_files:
    log_files = glob.glob("logs/*.yaml")

if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    print(f"Loading logs from: {latest_log}")
    
    trial_events = []
    with open(latest_log, 'r') as f:
        events = list(yaml.safe_load_all(f))
        for event in events:
            if event and event.get('event') == 'trial_complete':
                trial_events.append(event)
    
    trials_df = pd.DataFrame(trial_events)
    print(f"Loaded {len(trials_df)} trials.")
    
    if not trials_df.empty:
        # Calculate cumulative success rate per case
        trials_df['success'] = trials_df['is_correct'].astype(int)
        trials_df['cumulative_success_rate'] = trials_df.groupby('case_id')['success'].expanding().mean().reset_index(level=0, drop=True)
        
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=trials_df, x='trial_index', y='cumulative_success_rate', hue='case_id', alpha=0.7)
        plt.title('Cumulative Success Rate by Case')
        plt.xlabel('Trial Index')
        plt.ylabel('Success Rate')
        plt.show()
else:
    print("No log files found.")